---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    train.csv - the training set (all tickets issued 2004-2011)
    test.csv - the test set (all tickets issued 2012-2016)
    addresses.csv & latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32

In [1]:
import pandas as pd
import numpy as np

def blight_model():
    from sklearn.metrics import roc_auc_score
    from sklearn.ensemble import RandomForestRegressor

    #Insert lat and lon in address embbeded in adresses csv 
    add = pd.read_csv('addresses.csv') 
    lat = pd.read_csv('latlons.csv') 
    geoRef = pd.merge(add,lat, how='left', on='address')
    geoRef.set_index(['ticket_id'], inplace=True)
    geoRef=geoRef[['lat','lon']]

    train = pd.read_csv('train.csv', sep=',', encoding='cp1252', low_memory=False,
                        parse_dates = ['ticket_issued_date', 'hearing_date'], index_col='ticket_id')
    train = pd.merge(train, geoRef, how='left', left_index=True, right_index=True)

    test = pd.read_csv('test.csv', sep=',', encoding='cp1252', low_memory=False,
                        parse_dates = ['ticket_issued_date', 'hearing_date'], index_col='ticket_id')
    test = pd.merge(test, geoRef, how='left', left_index=True, right_index=True)

    train = train.dropna(subset=['compliance']) #drop compliance nan
    train = train[train['agency_name'] != 'Neighborhood City Halls'] #delete option
    #disposition feature (replace)
    disposition_replace = {'Responsible by Default': 'By default',
                           'Responsible by Determination': 'By determination',
                           'Responsible (Fine Waived) by Admis': 'Fine Waived',
                           'Responsible (Fine Waived) by Deter': 'Fine Waived',
                           'Responsible - Compl/Adj by Default': 'By default',
                           'Responsible - Compl/Adj by Determi': 'By determination',
                           'Responsible by Admission': 'By admission',
                           'Responsible by Dismissal': 'By default'}
    train.disposition.replace(disposition_replace, inplace=True)
    test.disposition.replace(disposition_replace, inplace=True)
    #discount feature (1 has discount, 0 without discount)
    train['discount'] = train.discount_amount.apply(lambda x:1 if x > 0 else 0)
    test['discount'] = test.discount_amount.apply(lambda x:1 if x > 0 else 0)
    #bins judgement amount
    cut = [-1, 100, 150, 300, float("inf")]
    train['judgment_level'] = pd.cut(train.judgment_amount, bins=cut)
    test['judgment_level'] = pd.cut(test.judgment_amount, bins=cut)
    #State replacement and missing values treatment
    states = ['AL','AK','AZ','AR','CA','CO','CT','DC','DE','FL','GA','HI','ID','IL','IN','IA','KS','KY','LA','ME',
              'MD','MA','MI','MN','MS','MO','MT','NE','NV','NH','NJ','NM','NY','NC','ND','OH','OK','OR','PA','RI',
              'SC','SD','TN','TX','UT','VT','VA','WA','WV','WI','WY']

    train['state_cat'] = train.state.apply(lambda x: x if x in states else 'NOT US')
    train.loc[(train.state_cat != 'NOT US') & (train.state_cat != 'MI'),'state_cat'] = 'NOT MI'
    train.loc[train.state.isnull(),'state_cat'] = 'MI' #Missing values most Frequent

    test['state_cat'] = test.state.apply(lambda x: x if x in states else 'NOT US')
    test.loc[(test.state_cat != 'NOT US') & (test.state_cat != 'MI'),'state_cat'] = 'NOT MI'
    test.loc[test.state.isnull(),'state_cat'] = 'MI' #Missing values most Frequent
    #Fill in loc and lat using the average in the most frequent places
    #lat and lon is filled with rhe average of 50 most frequencies locations
    mtrain = train.groupby(['lon','lat']).compliance.agg(['count', 'sum', 'mean', 'std']).sort_values('count',ascending=False)
    lon = mtrain[mtrain['count']>50].index.get_level_values(0).tolist()
    lat = mtrain[mtrain['count']>50].index.get_level_values(1).tolist()
    lon = round(sum(lon) / float(len(lon)),6)
    lat = round(sum(lat) / float(len(lat)),6)
    train.loc[train.lon.isnull(),'lon'] = lon #fill in missing values
    train.loc[train.lat.isnull(),'lat'] = lat #fill in missing values
    mtest = test.groupby(['lon','lat']).agency_name.agg(['count']).sort_values('count',ascending=False)
    lon = mtest[mtest['count']>50].index.get_level_values(0).tolist()
    lat = mtest[mtest['count']>50].index.get_level_values(1).tolist()
    lon = round(sum(lon) / float(len(lon)),6)
    lat = round(sum(lat) / float(len(lat)),6)
    test.loc[test.lon.isnull(),'lon'] = lon #fill in missing values
    test.loc[test.lat.isnull(),'lat'] = lat #fill in missing values
    #X and y values
    ycol = ['compliance']
    xcol = ['state_cat','judgment_level','discount','disposition','agency_name','lat','lon']
    X = train[xcol]
    Xval = test[xcol]
    y = np.ravel(train[ycol].astype('int'))
    #dummy variables
    X2 = X.copy()
    X2['train'] = [1] * X.shape[0]
    #print(X.head())
    X2val = Xval.copy()
    X2val['train'] = [0] * Xval.shape[0]

    X_merged = pd.concat([X2, X2val], axis=0)
    X_merged = pd.get_dummies(X_merged)#.reset_index(drop=True)
    X = X_merged[X_merged.train == 1]
    Xval = X_merged[X_merged.train == 0]
    #model random forest
    model = RandomForestRegressor(random_state=8453, n_estimators=25, max_depth=15, max_features=10)
    model.fit(X, y)
    pred = model.predict(X)
    #print('AUCROC for random forests')
    #print(roc_auc_score(y,pred))
    m = pd.Series(model.predict(Xval), index=Xval.index, name='compliance', dtype='float32')
    return m

In [2]:
#blight_model()

AUCROC for random forests
0.8943301874402421


ticket_id
284932    0.034946
285362    0.009513
285361    0.043883
285338    0.045139
285346    0.045055
            ...   
376496    0.026914
376497    0.026914
376499    0.041623
376500    0.041623
369851    0.041815
Name: compliance, Length: 61001, dtype: float32